<hr>

# EXPERIMENTO YOLOv5_UAVDT_10

<hr>

# Especificações do environment e máquina

In [ ]:
%%script bash
echo "Hash key do commit YoloV5 que esta em uso:"
cd ../../Submodules/yolov5
echo $( git rev-parse --verify HEAD)
echo
echo "Verificar localizacao do python sendo executado:"
which python
echo
echo "Conda Environment Utilizado: "
conda env export

In [ ]:
# ESPECIFICAÇÕES DA MÁQIUINA E DO EXPERIMENTO
import datetime
start_time = datetime.datetime.now()
from platform import uname
from os import environ, getcwd, sep
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

time_stamp = start_time.strftime("%d_%b_%Y_%Hh_%Mm")
time_zone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzname()
print("Horário: ", datetime.datetime.now(), "UTC", time_zone )
username = environ['USER']
sysinfo = uname()
hostname = sysinfo.node
root = getcwd()

experimento = root.split(sep)[-1].split("_")[:-1]
experimento = "_".join(experimento)
exp_number = root.split(sep)[-1].split("_")[-1]

# DADOS DE OUTPUT
exp_folder = f"{experimento}_{exp_number}-{time_stamp}"
output_dir_dropbox = f"~/Dropbox/Experimentos/{experimento}_{exp_number}/{exp_folder}"
output_dir_gcloud = f"gs://cfdy/Experimentos/{experimento}_{exp_number}/YOLOv5_UAVDT_train/{exp_folder}"
output_dir_log_txt = f"gs://cfdy/Experimentos_logs_txt/{experimento}_{exp_number}/logs"
output_dir_log_ipynb = f"gs://cfdy/Experimentos/{experimento}_{exp_number}/logs"

# DADOS DE TREINAMENTO:
tr = {
"epocas" : "50",
"model" : "yolov5l",
"img_resolution" : "640 640",
"batch_size" : "8",
"gpu_ids" : "0",
"logdir" : "../runs",
"num_workers" : "4",
"optimizer" : "--adam", # Following paths are relative to yolov5_w.
"log_artifacts" : "--log-artifacts", #Weight e Biases log
"project" : f"../{experimento}_train", # Save training session info
"name" : f"{exp_folder}"} # Save to "../{project}/{name}"
tr["model_yaml"] = f"./models/{tr['model']}.yaml" # relative to yolov5 folder
tr["weights"] = f"../../YOLOv5_UAVDT_6/YOLOv5_UAVDT_train/"+\
                 "YOLOv5_UAVDT_6-26_Feb_2021_04h_25m/weights/best.pt"
tr["hyperparameters"] = f"../hyp.finetune.yaml"

output_dir_local = f"{root}/{tr['project'][3:]}/{tr['name']}"

# Checar conda environment
conda_env = !printenv CONDA_PROMPT_MODIFIER
#if conda_env != ['(yv5r4) ']:
#    raise SystemExit("ERRO.. Conda environment incorreto.")

print(f"""
==Computer Specifications==
username: {username}

hostname: {hostname}

system info: {sysinfo}

{{root}}: {root}

conda env: {conda_env}

experimento: {experimento}

time stamp: {time_stamp}

time zone : UTC {time_zone}

output dropbox dir: {output_dir_dropbox}

output gcloud dir: {output_dir_gcloud}

output local dir: {output_dir_local}

Dados de treinamento: {tr}
===========================
""")

In [ ]:
# Formatação do comando para execução via nbconvert:
command = f"""conda activate yv5r4; mkdir logs; experimento="{experimento}_{exp_number}_TRAIN"; outputFileName=$(date "+"$experimento"_ipynb_%d_%B_%Y_%Hh_%Mm_%Ss"); echo "Salvando log de execucao em: $outputFileName.txt"; nohup jupyter nbconvert --to notebook --execute --allow-errors --output "./logs/$outputFileName" --ExecutePreprocessor.timeout=-1 --Application.log_level=10 $experimento.ipynb &> "./logs/$outputFileName.txt" &"""

!echo '{command}' > nbconvert_{experimento}_{exp_number}.sh

In [ ]:
# Verificar GPUs disponíveis
!nvidia-smi

<hr>

# Clonar o Repositório de Trabalho YOLOv5_w

In [ ]:
#Ir para a raiz
%cd {root}

In [ ]:
%%script bash
pwd
#Criar pasta de logs e runs
mkdir logs
#Remover diretórios antigos do yolov5_w
rm -rf yolov5_w
#Copiar diretório novo da pasta submodules (Sem Links)
cp -a ../../Submodules/yolov5_w/ yolov5_w/
ls

# Executar Script para Transformar Imagens do UAVDT Dataset para o formato requerido pelo YOLOv5

In [ ]:
# Apagar os links existentes para o dataset
!rm -rf UAVDT_YOLOv5_Dataset
# Criar novos hard links para o dataset e organizar o dataset no formato exigido pelo yolov5
!mkdir UAVDT_YOLOv5_Dataset
!python ../../utils/uavdt_to_yolov5.py --conjunto treinoEteste --dest ../{experimento}_{exp_number}/UAVDT_YOLOv5_Dataset --uavdt-path ../../Datasets/UAVDT --nosymlink

# Pré-processar as imagens

In [ ]:
movies_treino = ["M0101", "M0201", "M0202", "M0204", \
    "M0206", "M0207", "M0210", "M0301", "M0401", "M0402", \
    "M0501", "M0603","M0604", "M0605", "M0702", "M0703", \
    "M0704", "M0901", "M0902", "M1002", "M1003", "M1005", \
    "M1006", "M1008", "M1102", "M1201", "M1202", "M1304", \
    "M1305", "M1306"]

movies_teste = ["M0203", "M0205", "M0208", "M0209", "M0403", "M0601", \
        "M0602", "M0606", "M0701", "M0801", "M0802", "M1001", "M1004", \
        "M1007", "M1009", "M1101", "M1301", "M1302", "M1303", "M1401"]

movies = movies_treino + movies_teste
    
for movie_name in movies:
    print(f"[INFO] Analisando a sequência de imagens {movie_name}.")

    frame_boxes = {}
    with open(f"../../Datasets/UAVDT/UAV-benchmark-MOTD_v1.0/GT/{movie_name}_gt_ignore.txt") as igfile:
        lines = igfile.readlines()
        for line in lines:
            line = line.split(",")
            frame = int(line[0])
            x = int(line[2])
            y = int(line[3])
            w = int(line[4])
            h = int(line[5])
            classe = int(line[-1])
            if frame in frame_boxes:
                frame_boxes[frame] += [[frame, x, y, w, h, classe]]
            else:
                frame_boxes[frame] = [[frame, x, y, w, h, classe]]

    # Para cada vídeo, ajustar as imagens do UAVDT Local com os devidos blurs..
    conjunto = "train" if movie_name in movies_treino else "test"
    
    for frame, boxes in frame_boxes.items():
        img_path = f"./UAVDT_YOLOv5_Dataset/{conjunto}/images/{movie_name}_img{frame:06}.jpg"
        img = np.array(Image.open(img_path))#cv2.imread(img_path)
        for _,x,y,w,h,_ in boxes:
            img[y:y+h, x:x+w] = (0,0,0) #Black box
        Image.fromarray(img).save(img_path)

In [ ]:
# Checar efeitos do pré-processamento em uma "ignore area" de uma imagem qualquer do dataset.

img_path = f"./UAVDT_YOLOv5_Dataset/train/images/M0101_img000001.jpg"
img = Image.open(img_path)
# summarize some details about the image
print(img.format)
print(img.size)
print(img.mode)
# plot image
plt.figure(figsize=(14,10))
plt.imshow(img)

# Criar um data.yaml conforme exigido pelo yolov5


In [ ]:
# O conteúdo original não está adequado... mais fácil que editar: removerei-o e criarei um novo.

datayaml = """# train and val datasets (image directory or *.txt file with image paths)

train: ../UAVDT_YOLOv5_Dataset/train/images/
val: ../UAVDT_YOLOv5_Dataset/test/images/

# number of classes
nc: 3

# class names
names: ['car', 'truck', 'bus']
"""

%rm data.yaml
with open('data.yaml', 'w') as file:
    file.write(datayaml)
%cat data.yaml

# Executar Treinamento por 50 Eṕocas

In [ ]:
%cd {root}/yolov5_w
start_training_time = datetime.datetime.now()

!python train.py --epochs {tr["epocas"]} --cfg {tr["model_yaml"]} --weights {tr["weights"]}  --data ../data.yaml --img {tr["img_resolution"]} --batch-size {tr["batch_size"]} --device {tr["gpu_ids"]} --workers {tr["num_workers"]} {tr["optimizer"]} {tr["log_artifacts"]} --project {tr["project"]} --name {tr["name"]} --hyp {tr["hyperparameters"]}

finish_training_time = datetime.datetime.now()

# Salvar Resultados no Dropbox e no Google Cloud
Nesta etapa, salvarei os arquivos importantes no dropbox.

In [ ]:
%cd {root}
print("COPY TO DROPBOX")
%mkdir -p {output_dir_dropbox}
%cp -avl logs/ {output_dir_dropbox}
%cp -avl {output_dir_local}/ {output_dir_dropbox}

In [ ]:
print("COPY TO GCLOUD")
#logs/*.ipynb
!gsutil -m cp -r logs/*.ipynb {output_dir_log_ipynb}
#logs/*.txt
!gsutil -m cp -r logs/*.txt {output_dir_log_txt}
#train files
!gsutil -m cp -r {output_dir_local}/ {output_dir_gcloud}

# Finalização

In [ ]:
finish_time = datetime.datetime.now()
delta_time = finish_time - start_time
delta_time_train = finish_training_time - start_training_time
print(f"""
Notebook:
Start: {start_time}
Finish: {finish_time}
Duration (hour:min:sec.microsec):  {delta_time}

Train:
Start: {start_training_time}
Finish: {finish_training_time}
Duration (hour:min:sec.microsec):  {delta_time_train}""")

In [ ]:
print("Limpar arquivos desnecessários.")
%cd {root}
%rm -rf UAVDT_YOLOv5_Dataset yolov5_w data.yaml